# Erstellen einer FHIR Patient Ressource mit dem Python fhirclient
Um den Python fhirclient zu installieren kann man entweder die aktuellste Version aus dem [GitHub Repository](https://github.com/smart-on-fhir/client-py) nehmen oder die zuletzt auf [PyPi](https://pypi.org/project/fhirclient/) veröffentlichte Version. Da diese jedoch noch STU-3 ist, nutzen wir für die folgenden Beispiele die GitHub Version (R4).

In [ ]:
# Zelle wird nur für das Ausführen in Umgebungen wie Binder benötigt
!pip install git+https://github.com/smart-on-fhir/client-py.git

## IFrame wird hier nur zu Darstellungszwecken importiert und wird nicht für die späteren Aufgaben benötigt.

In [1]:
from IPython.display import IFrame

In [2]:
# Patient Ressource in der FHIR Spezifikation.
IFrame('https://hl7.org/fhir/patient.html#resource', width=1200, height=330)

In [4]:
from fhirclient.models import patient

In [5]:
# Instanziieren eines Patienten
my_patient = patient.Patient()

In [6]:
# help() Funktion, um mehr Informationen zu erhalten
help(my_patient)

Help on Patient in module fhirclient.models.patient object:

class Patient(fhirclient.models.domainresource.DomainResource)
 |  Patient(jsondict=None, strict=True)
 |  
 |  Information about an individual or animal receiving health care services.
 |  
 |  Demographics and other administrative information about an individual or
 |  animal receiving care or other health-related services.
 |  
 |  Method resolution order:
 |      Patient
 |      fhirclient.models.domainresource.DomainResource
 |      fhirclient.models.resource.Resource
 |      fhirclient.models.fhirabstractresource.FHIRAbstractResource
 |      fhirclient.models.fhirabstractbase.FHIRAbstractBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, jsondict=None, strict=True)
 |      Initialize all valid properties.
 |      
 |      :raises: FHIRValidationError on validation errors, unless strict is False
 |      :param dict jsondict: A JSON dictionary to use for initialization
 |      :param bool 

## Mit der Patient Ressource vertraut machen
Um erstmal einen Überblick davon zu erhalten wie die Patienten Ressource aufgebaut ist, bzw. welche Attribute diese enthält kann die Funktion **elementProperties()** aufgerufen werden, diese gibt dann alle Attribute der fhirclient Klasse **Patient()** mit ihren jeweiligen Eigenschaften zurück.

In [7]:
for property in my_patient.elementProperties():
    print(property)

('id', 'id', <class 'str'>, False, None, False)
('implicitRules', 'implicitRules', <class 'str'>, False, None, False)
('language', 'language', <class 'str'>, False, None, False)
('meta', 'meta', <class 'fhirclient.models.meta.Meta'>, False, None, False)
('contained', 'contained', <class 'fhirclient.models.resource.Resource'>, True, None, False)
('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('text', 'text', <class 'fhirclient.models.narrative.Narrative'>, False, None, False)
('active', 'active', <class 'bool'>, False, None, False)
('address', 'address', <class 'fhirclient.models.address.Address'>, True, None, False)
('birthDate', 'birthDate', <class 'fhirclient.models.fhirdate.FHIRDate'>, False, None, False)
('communication', 'communication', <class 'fhirclient.models.patient.PatientCommunication'>, True, None, False)
('contact',

## Name des Patienten / der Patientin

In [8]:
IFrame('https://hl7.org/fhir/datatypes.html#HumanName', width=1200, height=330)

In [9]:
from fhirclient.models import humanname

In [10]:
name = humanname.HumanName()
name.family = 'Bach'
name.given = ['Johann', 'Sebastian']
name.text = 'Johann Sebastian Bach'
name.use = 'official'
my_patient.name = [name]

## Geburtsdatum

In [11]:
IFrame('https://hl7.org/fhir/datatypes.html#date', width=1200, height=330)

In [12]:
from fhirclient.models import fhirdate

In [13]:
date_of_birth = fhirdate.FHIRDate('1967-05-25')
my_patient.birthDate = date_of_birth

In [14]:
import json

In [15]:
print(json.dumps(my_patient.as_json(), indent=4))

{
    "birthDate": "1967-05-25",
    "name": [
        {
            "family": "Bach",
            "given": [
                "Johann",
                "Sebastian"
            ],
            "text": "Johann Sebastian Bach",
            "use": "official"
        }
    ],
    "resourceType": "Patient"
}


## Pythons Requests Library
Die [Requests Library](https://requests.readthedocs.io/en/master/) enthält Funktionen um HTTP Anfragen schnell um simpel an einen Server zu schicken. 

In [16]:
import requests

In [23]:
# fhir_test_server = 'https://server.fire.ly'
fhir_test_server = 'http://hapi.fhir.org/baseR4'


headers = {
'Accept':'application/fhir+json; fhirVersion=4.0',
'Content-Type':'application/fhir+json; fhirVersion=4.0'
}

In [24]:
response = requests.post(f'{fhir_test_server}/Patient/$validate', headers = headers, data = json.dumps(my_patient.as_json()))
operation_outcome = response.json()

In [25]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

In [26]:
print(json.dumps(operation_outcome, indent=4))

{
    "resourceType": "OperationOutcome",
    "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><h1>Operation Outcome</h1><table border=\"0\"><tr><td style=\"font-weight: bold;\">WARNING</td><td>[Patient, Line 1, Col 2]</td><td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>\n\t\t\t</tr>\n\t\t</table>\n\t</div>"
    },
    "issue": [
        {
            "severity": "warning",
            "code": "processing",
            "diagnostics": "dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed",
            "location": [
                "Patient",
                "Line 1, Col 2"
            ]
        }
    ]
}


In [27]:
response = requests.post(f'{fhir_test_server}/Patient', headers = headers, data = json.dumps(my_patient.as_json()))
operation_outcome = response.json()

In [28]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

In [29]:
print(json.dumps(operation_outcome, indent=4))

{
    "resourceType": "Patient",
    "id": "3259700",
    "meta": {
        "versionId": "1",
        "lastUpdated": "2022-06-15T14:24:32.349+00:00"
    },
    "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Johann Sebastian <b>BACH </b></div><table class=\"hapiPropertyTable\"><tbody><tr><td>Date of birth</td><td><span>25 May 1967</span></td></tr></tbody></table></div>"
    },
    "name": [
        {
            "use": "official",
            "text": "Johann Sebastian Bach",
            "family": "Bach",
            "given": [
                "Johann",
                "Sebastian"
            ]
        }
    ],
    "birthDate": "1967-05-25"
}
